<a href="https://colab.research.google.com/github/kai054631/Weather_Data/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from pandas import read_csv
from sklearn.model_selection import GridSearchCV,KFold, cross_val_score,train_test_split as split

from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

Read dataset from github and show frist 5 rows

In [3]:
df = read_csv("https://raw.githubusercontent.com/Des282/Dataset/refs/heads/main/seattle-weather.csv")#store and read
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


from matplotlib import pyplot as plt
_df_0['precipitation'].plot(kind='hist', bins=20, title='precipitation')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['temp_max'].plot(kind='hist', bins=20, title='temp_max')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['temp_min'].plot(kind='hist', bins=20, title='temp_min')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['wind'].plot(kind='hist', bins=20, title='wind')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['weather'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='precipitation', y='weather', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['date'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='temp_max', y='date', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['weather'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='temp_max', y='weather', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

check if any data is missing

In [4]:
df.isna().sum()

,0
date,0
precipitation,0
temp_max,0
temp_min,0
wind,0
weather,0


show the size of dataset

In [5]:
df.shape

(1461, 6)

In [6]:
df = df.drop(columns = ["date"])
X = df.drop(columns=["weather"])
y = df["weather"]

In [7]:
X.shape

(1461, 4)

Train and evaluate at least five different machine learning algorithms, such as:
• Linear Regression/Logistic Regression    
• Decision Trees     
• Random Forest    
• Support Vector Machines (SVM)    
• k-Nearest Neighbors (kNN)    
• Gradient Boosting Trees   
• Neural Networks   

In [16]:
#Use spot-checking to quickly evaluate the performance of 8 machine learning algorithms
models = {}
models['lgr'] = LR()
models['knn'] = KNN()
models['gnb'] = GNB()
models['svc'] = SVC()
models['dtc'] = DTC()
models['rfc'] = RFC()
models['gbc'] = GBC()
models['mlp'] = MLP()


lgr: 84.326%, 0.968%
knn: 75.838%, 1.078%
gnb: 84.326%, 1.355%
svc: 77.481%, 1.078%
dtc: 75.291%, 1.091%
rfc: 83.025%, 0.844%
gbc: 82.341%, 0.933%
mlp: 84.326%, 0.774%


In [33]:
#Hyperparameter tuning
for i in range(0):
  kf = KFold(n_splits=i, shuffle=True, random_state=42) #set k=3 fold cross validation
  print(f'Kfold Value={i}')
  for n in models:
    scores = cross_val_score(models[n], X, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"{n}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy


In [9]:
Robust_scl = RobustScaler()#Feature Scaling with RobustScaler
Xs1 = Robust_scl.fit_transform(X)

In [10]:
MinMax_scl = MinMaxScaler()#Feature Scaling with MinMaxScaler
Xs2 = MinMax_scl.fit_transform(X)

In [11]:
Standard_scl = StandardScaler()#Feature Scaling with StandardScaler
Xs3 = Standard_scl.fit_transform(X)

In [17]:
print(f"Feature Scaling with")
for n in models:
    scores = cross_val_score(models[n], Xs1, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"ROBUST SCALING {n}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for i in models:
    scores = cross_val_score(models[i], Xs2, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"MINMAX SCALING {i}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for j in models:
    scores = cross_val_score(models[j], Xs3, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"STANDARD SCALING {j}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy


Feature Scaling with
ROBUST SCALING lgr: 81.588%, 0.512%
ROBUST SCALING knn: 77.892%, 0.512%
ROBUST SCALING gnb: 84.326%, 1.355%
ROBUST SCALING svc: 78.371%, 1.116%
ROBUST SCALING dtc: 75.633%, 1.426%
ROBUST SCALING rfc: 83.162%, 0.168%
ROBUST SCALING gbc: 82.204%, 0.844%
ROBUST SCALING mlp: 85.079%, 0.422%
 
MINMAX SCALING lgr: 73.238%, 0.860%
MINMAX SCALING knn: 70.363%, 0.635%
MINMAX SCALING gnb: 84.326%, 1.355%
MINMAX SCALING svc: 75.565%, 0.290%
MINMAX SCALING dtc: 75.633%, 0.756%
MINMAX SCALING rfc: 82.957%, 0.731%
MINMAX SCALING gbc: 82.272%, 1.024%
MINMAX SCALING mlp: 76.523%, 0.792%
 
STANDARD SCALING lgr: 78.166%, 0.678%
STANDARD SCALING knn: 71.458%, 1.099%
STANDARD SCALING gnb: 84.326%, 1.355%
STANDARD SCALING svc: 77.550%, 0.792%
STANDARD SCALING dtc: 75.017%, 0.968%
STANDARD SCALING rfc: 83.025%, 0.097%
STANDARD SCALING gbc: 82.341%, 0.933%
STANDARD SCALING mlp: 81.177%, 0.827%


In [18]:
#First method: #4/2 = 2 reducing the feature by half
pca = PCA(n_components=2)
Xsr1 = pca.fit_transform(Xs1)
Xsr2 = pca.fit_transform(Xs2)
Xsr3 = pca.fit_transform(Xs3)

In [19]:
print(f"reducing the feature by half")
for n in models:
    scores = cross_val_score(models[n], Xsr1, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"ROBUST SCALING {n}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for i in models:
    scores = cross_val_score(models[i], Xsr2, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"MINMAX SCALING {i}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for j in models:
    scores = cross_val_score(models[j], Xsr3, y, cv=kf, n_jobs=-1) #get the accuracy
    print(f"STANDARD SCALING {j}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy

reducing the feature by half
ROBUST SCALING lgr: 78.850%, 0.290%
ROBUST SCALING knn: 77.139%, 0.512%
ROBUST SCALING gnb: 78.029%, 0.605%
ROBUST SCALING svc: 78.508%, 0.539%
ROBUST SCALING dtc: 69.815%, 2.256%
ROBUST SCALING rfc: 77.550%, 1.141%
ROBUST SCALING gbc: 78.439%, 0.335%
ROBUST SCALING mlp: 80.082%, 0.444%
 
MINMAX SCALING lgr: 62.423%, 2.012%
MINMAX SCALING knn: 59.343%, 1.266%
MINMAX SCALING gnb: 61.875%, 1.943%
MINMAX SCALING svc: 65.024%, 1.116%
MINMAX SCALING dtc: 55.578%, 2.373%
MINMAX SCALING rfc: 61.328%, 1.521%
MINMAX SCALING gbc: 63.107%, 0.512%
MINMAX SCALING mlp: 64.408%, 1.258%
 
STANDARD SCALING lgr: 65.092%, 1.432%
STANDARD SCALING knn: 63.587%, 1.884%
STANDARD SCALING gnb: 62.423%, 1.006%
STANDARD SCALING svc: 66.598%, 0.953%
STANDARD SCALING dtc: 57.016%, 1.258%
STANDARD SCALING rfc: 63.792%, 0.194%
STANDARD SCALING gbc: 65.366%, 1.365%
STANDARD SCALING mlp: 67.146%, 0.768%


In [20]:
#Second method: create new features based on the existing features
win_size = 4
df['mean_precipitation'] = df['precipitation'].rolling(win_size).mean()
df['mean_temp_max'] = df['temp_max'].rolling(win_size).mean()
df['mean_temp'] = df['temp_max'] + df['temp_min'] / 2
df['mean_temp_min'] = df['temp_min'].rolling(win_size).mean()
df['mean_wind'] = df['wind'].rolling(win_size).mean()

print(df.shape)
print(df.head(10))

(1461, 10)
   precipitation  temp_max  temp_min  wind  weather  mean_precipitation  \
0            0.0      12.8       5.0   4.7  drizzle                 NaN   
1           10.9      10.6       2.8   4.5     rain                 NaN   
2            0.8      11.7       7.2   2.3     rain                 NaN   
3           20.3      12.2       5.6   4.7     rain               8.000   
4            1.3       8.9       2.8   6.1     rain               8.325   
5            2.5       4.4       2.2   2.2     rain               6.225   
6            0.0       7.2       2.8   2.3     rain               6.025   
7            0.0      10.0       2.8   2.0      sun               0.950   
8            4.3       9.4       5.0   3.4     rain               1.700   
9            1.0       6.1       0.6   3.4     rain               1.325   

   mean_temp_max  mean_temp  mean_temp_min  mean_wind  
0            NaN       15.3            NaN        NaN  
1            NaN       12.0            NaN        N

In [21]:
df = df.dropna()
df.head()

,precipitation,temp_max,temp_min,wind,weather,mean_precipitation,mean_temp_max,mean_temp,mean_temp_min,mean_wind
3,20.3,12.2,5.6,4.7,rain,8.000,11.825,15.0,5.15,4.050
4,1.3,8.9,2.8,6.1,rain,8.325,10.850,10.3,4.60,4.400
5,2.5,4.4,2.2,2.2,rain,6.225,9.300,5.5,4.45,3.825
6,0.0,7.2,2.8,2.3,rain,6.025,8.175,8.6,3.35,3.825
7,0.0,10.0,2.8,2.0,sun,0.950,7.625,11.4,2.65,3.150


In [22]:
X_New = df.drop(columns=["weather"])
y_New = df["weather"]

Xs1 = Robust_scl.fit_transform(X_New)
Xs2 = MinMax_scl.fit_transform(X_New)
Xs3 = Standard_scl.fit_transform(X_New)

In [23]:
print(X_New.shape)
print(y_New.shape)

(1458, 9)
(1458,)


In [24]:
print(f"reate new features based on the existing features")

for n in models:
    scores = cross_val_score(models[n], Xs1, y_New, cv=kf, n_jobs=-1) #get the accuracy
    print(f"ROBUST SCALING {n}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for i in models:
    scores = cross_val_score(models[i], Xs2, y_New, cv=kf, n_jobs=-1) #get the accuracy
    print(f"MINMAX SCALING {i}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for j in models:
    scores = cross_val_score(models[j], Xs3, y_New, cv=kf, n_jobs=-1) #get the accuracy
    print(f"STANDARD SCALING {j}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy

reate new features based on the existing features
ROBUST SCALING lgr: 81.001%, 2.695%
ROBUST SCALING knn: 74.623%, 1.561%
ROBUST SCALING gnb: 76.406%, 1.851%
ROBUST SCALING svc: 78.532%, 3.063%
ROBUST SCALING dtc: 78.189%, 1.333%
ROBUST SCALING rfc: 85.802%, 1.848%
ROBUST SCALING gbc: 84.705%, 1.093%
ROBUST SCALING mlp: 83.608%, 2.193%
 
MINMAX SCALING lgr: 73.045%, 1.268%
MINMAX SCALING knn: 69.890%, 0.513%
MINMAX SCALING gnb: 76.406%, 1.851%
MINMAX SCALING svc: 74.143%, 1.588%
MINMAX SCALING dtc: 78.121%, 1.119%
MINMAX SCALING rfc: 85.391%, 1.981%
MINMAX SCALING gbc: 84.636%, 1.305%
MINMAX SCALING mlp: 75.995%, 1.358%
 
STANDARD SCALING lgr: 78.601%, 2.310%
STANDARD SCALING knn: 70.850%, 0.829%
STANDARD SCALING gnb: 76.406%, 1.851%
STANDARD SCALING svc: 75.446%, 1.765%
STANDARD SCALING dtc: 78.464%, 2.193%
STANDARD SCALING rfc: 85.391%, 1.770%
STANDARD SCALING gbc: 84.705%, 1.238%
STANDARD SCALING mlp: 79.904%, 2.637%


In [28]:
X_New = df.drop(columns=["mean_temp"]) #delete column of mean value of temperature
X_New.shape

(1458, 9)

In [29]:
Xs1 = Robust_scl.fit_transform(X_New)
Xs2 = MinMax_scl.fit_transform(X_New)
Xs3 = Standard_scl.fit_transform(X_New)

ValueError: could not convert string to float: 'rain'

In [ ]:
for n in models:
    scores = cross_val_score(models[n], Xs1, y_New, cv=kf, n_jobs=-1) #get the accuracy
    print(f"ROBUST SCALING {n}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for i in models:
    scores = cross_val_score(models[i], Xs2, y_New, cv=kf, n_jobs=-1) #get the accuracy
    print(f"MINMAX SCALING {i}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy
print(" ")
for j in models:
    scores = cross_val_score(models[j], Xs3, y_New, cv=kf, n_jobs=-1) #get the accuracy
    print(f"STANDARD SCALING {j}: {scores.mean():.3%}, {scores.std():.3%}")#get the mean and standard deviation of accuracy

ROBUST SCALING lgr: 81.001%, 2.695%
ROBUST SCALING knn: 74.623%, 1.561%
ROBUST SCALING gnb: 76.406%, 1.851%
ROBUST SCALING svc: 78.532%, 3.063%
ROBUST SCALING dtc: 77.709%, 1.192%
ROBUST SCALING rfc: 85.460%, 1.962%
ROBUST SCALING gbc: 84.842%, 1.180%
ROBUST SCALING mlp: 83.745%, 2.098%
 
MINMAX SCALING lgr: 73.045%, 1.268%
MINMAX SCALING knn: 69.890%, 0.513%
MINMAX SCALING gnb: 76.406%, 1.851%
MINMAX SCALING svc: 74.143%, 1.588%
MINMAX SCALING dtc: 76.955%, 1.770%
MINMAX SCALING rfc: 85.802%, 2.184%
MINMAX SCALING gbc: 84.705%, 1.399%
MINMAX SCALING mlp: 76.132%, 0.935%
 
STANDARD SCALING lgr: 78.601%, 2.310%
STANDARD SCALING knn: 70.850%, 0.829%
STANDARD SCALING gnb: 76.406%, 1.851%
STANDARD SCALING svc: 75.446%, 1.765%
STANDARD SCALING dtc: 78.395%, 1.908%
STANDARD SCALING rfc: 86.008%, 1.817%
STANDARD SCALING gbc: 84.842%, 1.272%
STANDARD SCALING mlp: 79.492%, 2.025%
